In [ ]:
# Imports necessary modules
from suds.client import Client
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import datetime
import time

# Imports matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

# Data trimming function
def acerta_data(data):
    try: 
        a = datetime.datetime.strptime(data, '%d/%m/%Y').strftime('%m/%d/%Y')
    except:    
        pass
    try:
        a = datetime.datetime.strptime(data, '%m/%Y')
        if a.month == 12:
            a = datetime.date(a.year, a.month, 31).strftime('%m/%d/%Y')
        else:
            a = datetime.date(a.year, a.month+1, a.day) - datetime.timedelta(days=1)
            a = a.strftime('%m/%d/%Y')
    except:
        pass
    try:    
        a = datetime.date(int(data), 12, 31).strftime('%m/%d/%Y')
    except:
        pass
    return a

# Loads series indexes for lookup
series_bcb = pd.read_csv('~/Downloads/cotacoes.txt', index_col=0, header=0)
# Lookup expression
#   series_bcb[series_bcb['Descrição'].str.contains("Dólar")==True]

print('Acesso à API do Banco Central do Brasil (v. 0.1)')

# User input
series_selecionadas = []
# String searching on the database
while True:
    search_string = input("\nDigite parte do nome da série que deseja (p.ex. 'dólar')\nou FIM para prosseguir: ")
    if search_string == 'FIM':
        break
    else:
        result = series_bcb[series_bcb['Descrição'].str.contains(search_string, case=False)==True]
        result.insert(0,'Cód',result.index)
        result.index = range(1, len(result.index)+1)
        print(result['Descrição'])
        # User selection
        series_input = []
        series_input.extend(list(int(x) for x in input("\nDigite o número das séries que deseja (separe com um espaço): ").split()))
        # Transforming user input in usable indexes
        for n in range(len(series_input)):
            series_input[n] = result.get_value(series_input[n],'Cód')
        series_selecionadas.extend(series_input)

series_selecionadas_nom = {}
for x in series_selecionadas:
    series_selecionadas_nom[x] = series_bcb.get_value(x, 'Descrição')
    
datainicio = str(input("\nData inicial (DD/MM/AAAA): ") or '01/09/2014')
datafim = str(input("Data final (DD/MM/AAAA): ") or '01/09/2016')
nome_output = input("Nome do arquivo de saída: ") or 'BCB_SGS'

# Creates Pandas DataFrame with calandar days as index
index = pd.date_range(datetime.datetime.strptime(datainicio, '%d/%m/%Y').strftime('%m/%d/%Y'), datetime.datetime.strptime(datafim, '%d/%m/%Y').strftime('%m/%d/%Y'))
df = pd.DataFrame(index=index, columns=series_selecionadas)

# Activates WSDL and retrieves time series
url = 'https://www3.bcb.gov.br/sgspub/JSP/sgsgeral/FachadaWSSGS.wsdl'
bcb = Client(url)
try:
    for n in range(len(series_selecionadas)):
        print('\nBuscando série ', repr(series_selecionadas_nom[series_selecionadas[n]]),'.')
        dados = bcb.service.getValoresSeriesXML([series_selecionadas[n]], datainicio, datafim)
        # Parse XML data
        root = ET.fromstring(dados)
        # Updates the DataFrame values
        for serie in range(len(root)):
            for ponto in range(len(root[serie])):
                df.set_value(acerta_data(root[serie][ponto][0].text), int(root[serie].attrib['ID']), float(root[serie][ponto][1].text))
        df = df.apply(lambda x: pd.to_numeric(x, errors='ignore'))
        print('Aguardando 2 segundos...')
        time.sleep(2)

except Exception as detail:
    print(detail)

# Previous list generator
#   datas = []
#   for serie in range(len(root)):
#       for ponto in range(len(root[serie])):
#              datas.append([int(root[serie].attrib['ID']), root[serie][ponto][0].text, float(root[serie][ponto][1].text)])
# Equivalent comprehension
#   [(int(root[serie].attrib['ID']), root[serie][ponto][0].text, float(root[serie][ponto][1].text)) for serie in range(len(root)) for ponto in range(len(root[serie]))]

# Export to CSV file
df = df.dropna()
df = df.rename(columns=series_selecionadas_nom)
df.to_csv('~/Desktop/'+nome_output+'.csv')
print('\nArquivo {arquivo} gravado com sucesso.'.format(arquivo=nome_output))